In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [7]:
import torch
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import datasets
from datasets import Dataset, DatasetDict
from datasets import ClassLabel

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device

### получение предсказаний

In [5]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/rubert_finetuned",
    num_labels=3,
    id2label=id2label,
    label2id=label2id,
    attention_probs_dropout_prob=0.1,
    hidden_dropout_prob=0.1
)

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/rubert_finetuned")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [39]:
# train test split
df =  pd.read_csv("/content/drive/MyDrive/data_sentiment.csv", index_col = False, skiprows=0, encoding='utf-16')
df = df.rename(columns={'review': 'text', 'sentiment': 'label'})
df['text'].replace('', np.nan, inplace=True)
df = df.dropna()

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
test_texts = list(test_data['text'])
target = list(test_data['label'])

In [11]:
# сохраняю тестовый сет в файл
test_data.to_csv('2.csv')

In [12]:
# использую пайплайн из huggingface, получаю предсказания модели для всех предложений из теста
sa = pipeline(task='sentiment-analysis', model = model, tokenizer = tokenizer, max_length=512, truncation=True)
pred = []
for sentence in tqdm(test_texts):
  out = sa(sentence)
  pred.append(out)
  with open('pred.txt', 'a') as f:
    f.write(sentence + '\t' + str(out)+'\n')

  0%|          | 0/29411 [00:00<?, ?it/s]

In [75]:
# сохраняю тексты и предсказания в файл
dic = {'text':test_texts, 'label':pred}
output_df = pd.DataFrame(dic)
output_df.to_csv('output.tsv', sep = '\t')

In [42]:
# оставляю от предсказаний только label (до этого было то, что возвращала модель, типа "[{'label': 'neutral', 'score': 0.7671204209327698}]")
pred_for_f1 = []
for i in output_df['label']:
  #l = i[0]
  pred_for_f1.append(i[0]['label'])
pred_for_f1[:10]

neutral


TypeError: string indices must be integers

### f1 + достаю неверные предсказания для текстов с оценкой 1 или 5

In [22]:
f1 = f1_score(target, pred_for_f1, average=None)
f1

array([0.78116599, 0.71698614, 0.94252425])

In [45]:
f1 = f1_score(target, output_df['label'], average='micro')
f1

0.8490020740539252

In [21]:
# достаю предложения с 1 и 5 звездами из датасета Hotels
hotels_train = pd.read_excel('Training_data_marked.xlsx', sheet_name = 1)
hotels_test = pd.read_excel('Test_data_marked.xlsx', sheet_name = 0)
hotels = pd.concat([hotels_train, hotels_test], axis=0, ignore_index=True)

h1and5 = hotels[(hotels['Общая оценка'] == 1) | (hotels['Общая оценка'] == 5)] # оставляю только те, где оценка 1 или 5
h1and5 = h1and5.reset_index(drop=True)
h1and5

,Общая оценка,Цена/качество,Расположение,Качество сна,Номера,Чистота,Обслуживание,Текст отзыва
0,1,NaN,NaN,NaN,2.0,1.0,1.0,"Начнем с того, что в этом отеле не берут деньг..."
1,5,NaN,NaN,NaN,NaN,NaN,NaN,"Отель находится в отдалении от центра,но пешко..."
2,5,NaN,NaN,NaN,NaN,NaN,NaN,Приехали с сестрой и её мужем на машине и с со...
3,5,5.0,NaN,NaN,NaN,5.0,5.0,Чистота и удобство в номерах – по высшему клас...
4,5,4.0,4.0,5.0,5.0,5.0,5.0,В сети отелей NH Collection отдыхаем впервые. ...
...,...,...,...,...,...,...,...,...
30532,5,5.0,3.0,5.0,5.0,4.0,4.0,Этот апарт-отель предлагает гостям отличный ва...
30533,5,4.0,4.0,5.0,5.0,4.0,5.0,"Для нас было плюсом, что в этом отеле можно ос..."
30534,5,5.0,4.0,5.0,5.0,5.0,5.0,Очень понравился отель.Хорошие номера с кухней...
30535,5,5.0,4.0,5.0,5.0,5.0,4.0,"В Петербурге бываю очень часто, поэтому появле..."


In [113]:
# здесь нахожу предложения с изначальной оценкой 1 или 5, на которых модель ошиблась
wrong_sentences = []
for i in tqdm(range(len(output_df))): # иду по ответам модели
  text = output_df['text'][i]     # достаю текст отзыва
  if text in set(h1and5['Текст отзыва']):  # если отзыв есть во множестве "тексты из датасета Hotels"
    index = h1and5.index[h1and5['Текст отзыва']==text].tolist()[0]    # индекс строки с этим текстом в датасете Hotels
                                                                         # (чтобы потом достать (изначальный) label)

    # 4 следующие строчки: если (изначальная) оценка была 5, заменяю на positive, если 1 -- negativ
    # (чтобы можно было сравнить с лейблами, предсказанными моделью)
    if h1and5['Общая оценка'][index] == 5:
      label = 'positive'
    elif h1and5['Общая оценка'][index] == 1:
      label = 'negative'

    if label != output_df['label'][i]:      # если модель предсказала неверно
      wrong_sentences.append(text)

      with open('wrong.tsv', 'a') as f:    # пишу в файл: индекс текста в изначальном датасете (Hotels),
                                            # текст, правильный label, предсказанный label
        f.write(str(index) + '\t' + text +  '\t' + str(h1and5['Общая оценка'][index])
          + '\t' + output_df['label'][i] + '\n')

  0%|          | 0/29411 [00:00<?, ?it/s]

### достаю все не нейтральные предложения, на которых модель ошиблась

In [40]:
output_df = pd.read_csv('output_FINAL.tsv', sep = '\t', index_col = False)
output_df = output_df.drop(columns=['Unnamed: 0'])
output_df

,text,label
0,Лосин хватило на один день)),neutral
1,"ужасная ткань - полиэстер, прилипает к телу ...",negative
2,полный ужас!,negative
3,КАЧЕСТВО ОЧЕНЬ ХОРОШЕЕ.РАЗМЕРНАЯ СЕТКА СООТВЕТ...,neutral
4,Отель расположен в нескольких шагах от метро А...,positive
...,...,...
29406,честный продавец. хоть заказ и не пришёл (в ...,negative
29407,"Мой самый, самый, самый любимый отель!!!!!! Та...",positive
29408,Путешествовала с сыном подростком. Отель уютны...,positive
29409,"Трусы сделаны хорошо, к этому претензий нет. Н...",neutral


In [29]:
# это наш тестовый датасет, оставляю в нем только positive и negative
test_data = test_data.drop(columns=['Unnamed: 0'])

test_withoutneutral = test_data[(test_data['label'] == 'negative') | (test_data['label'] == 'positive')]
test_withoutneutral = test_withoutneutral.reset_index(drop=True)
test_withoutneutral

,text,label
0,Лосин хватило на один день)),negative
1,"ужасная ткань - полиэстер, прилипает к телу ...",negative
2,КАЧЕСТВО ОЧЕНЬ ХОРОШЕЕ.РАЗМЕРНАЯ СЕТКА СООТВЕТ...,positive
3,Отель расположен в нескольких шагах от метро А...,positive
4,для деловой поездки или просто шопинга - идеал...,positive
...,...,...
22033,Хороший отель с прекрасным расположением. Жили...,positive
22034,Для меня это лучший отель Питера. Уровень серв...,positive
22035,"Мой самый, самый, самый любимый отель!!!!!! Та...",positive
22036,Путешествовала с сыном подростком. Отель уютны...,positive


In [33]:
# здесь нахожу предложения с изначальной оценкой 1 или 5, на которых модель ошиблась
wrong_sentences2 = []
for i in tqdm(range(len(output_df))): # иду по ответам модели
  text = output_df['text'][i]     # достаю текст отзыва
  if text in set(test_withoutneutral['text']):  # если отзыв есть во множестве "не нейтральные тексты"
    index = test_withoutneutral.index[test_withoutneutral['text']==text].tolist()[0]    # индекс строки с этим текстом в датасете без нейтральных текстов
                                                                         # (чтобы потом достать (изначальный) label)



    if test_withoutneutral['label'][index] != output_df['label'][i]:      # если модель предсказала неверно
      text = text.replace('\n', ' ') #иногда в отзывах переносы строк, убираю, чтобы норм записать в файл
      wrong_sentences2.append(text)

      with open('wrong2.tsv', 'a') as f:    # пишу в файл: индекс текста в изначальном датасете (Hotels),
                                            # текст, правильный label, предсказанный label
        f.write(str(index) + '\t' + text +  '\t' + str(test_withoutneutral['label'][index])
          + '\t' + output_df['label'][i] + '\n')

  0%|          | 0/29411 [00:00<?, ?it/s]

In [49]:
wr = pd.read_csv('wrong1&5.tsv', sep='\t', index_col = False)
wr = wr.sort_values(by=['id'])
wr.to_csv('wrong_texts_1&5.tsv', sep = '\t', index=False)